In [ ]:
import os

spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,422 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.c

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-09-29 18:16:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-09-29 18:16:25 (1.26 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)

books_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
books_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-01-14 00:10:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [ ]:
# Count number of records
books_df.count()

3105520

In [ ]:
# Drop null values and count new total records
books_df = df.dropna()
books_df.count()

2794293

In [ ]:
# Print schema
books_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Create review dataframe to match review_id_table table
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RQ58W7SMO911M|   12076615| 385730586|     122662979|2005-01-14 00:10:00|
|  RF6IUKMGL8SF|   12703090| 811828964|      56191234|2005-01-14 00:10:00|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049|2005-01-14 00:10:00|
| RATOTLA3OF70O|   50732546| 373836635|     348672532|2005-01-14 00:10:00|
|R1TNWRKIVHVYOV|   51964897| 262181533|     598678717|2005-01-14 00:10:00|
|R2F53LI9KK9MOY|   31048862| 316769487|     862964341|2005-01-14 00:10:00|
|R1KJ6MB7MRSQFF|   53000124| 805076069|     145341889|2005-01-14 00:10:00|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054|2005-01-14 00:10:00|
|R1VE0FQQ0QTQJN|   48541186| 300108834|     915891133|2005-01-14 00:10:00|
|R1VKEE2NWSWDRU|   15280864| 446531081|     880645124|2005-01-14 00:10:00|
|R2UP6XSVYJBJ2H|   242092

In [ ]:
# Print schema
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Create product dataframe to match products table(only unique values)
from pyspark.sql.functions import asc
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df = products_df.orderBy(asc("product_id"))
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|        78|Conversations wit...|
|       116|Georgina Goodman ...|
|       868|Foot Rot of Piper...|
|     20214|Connoisseur's Han...|
|     24341|Groundwork of Eva...|
|     42382|      Other Shepards|
|    913154|The Way Things Wo...|
|   1006002|    The Rescue Party|
|   1018043| The Enchanted Horse|
|   1035649|Under Milk Wood: ...|
|   1046535|   As the Crow Flies|
|   1046810|Tess of the D'Urb...|
|   1047868|Kidnapped (Harper...|
|   1050818|Star Wars: The Tr...|
|   1052020|            Lord Jim|
|   1052942|      Yellow Admiral|
|   1053310|        Hms Surprise|
|   1053396|War Poems: An Ant...|
|   1053736|Sir Gawain and th...|
|   1053744| Pearl and Sir Orfeo|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create customer dataframe to match customers table
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50122160|         19923|
|   50732546|          9126|
|   52615377|          2379|
|   45041039|          2034|
|   50776149|          1642|
|   50913245|          1559|
|   45273033|          1457|
|   20595117|          1374|
|   39569598|          1263|
|   49693975|          1262|
|   36642996|          1203|
|   38491967|          1196|
|   50881246|          1152|
|   52254603|          1046|
|   49750558|           946|
|   49115690|           946|
|   51987970|           945|
|   50608834|           919|
|   51147790|           911|
|   43546353|           902|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create vine dataframe to match vine_table table

from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# # Filter for only columns from Amazon's Vine program
# vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|
|R2XIM9LT335WHE|          4|            3|         10|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|
|R2UP6XSVYJBJ2H|          5|            0|          0|   N|
|R21SYDQ70ILUC0|          5|            1|          1|   N|
| R7M06Z88PD7SX|          4|           14|         14|   N|
| RRS38KZ4WB5O2|          5|            

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://database-1.c1lahiassvlb.us-west-1.rds.amazonaws.com:5432/book_reviews"
config = {"user":"postgres",
          "password": "March594!",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_id_df table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.3.0-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.3.0-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_table table in RDS

vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.3.0-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.3.0-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored